Create .env file and then make sure that it has following variables
```
AZURE_OPENAI_API_KEY="xxx"
AZURE_OPENAI_ENDPOINT="https://xxx.openai.azure.com/"
AZURE_OPENAI_DEPLOYMENT="gpt-4o-2024-08-06"
API_NINJAS_API_KEY="xxx"
```

Note:
- Use https://api-ninjas.com/ to get API_NINJAS_API_KEY. Use monthly free subscription at minimum
- Use Azure OpenAI resource to get AZURE_OPENAI_ variables.

# Create Azure OpenAI Client

In [1]:
from autogen_ext.models import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os
import requests

In [2]:
# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')
API_NINJAS_API_KEY=os.getenv('API_NINJAS_API_KEY')

In [3]:
client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True,
    }
)

# Define function to get temperature for a given city

In [4]:
async def get_temp(city: str) -> str:

    # First get the latitude and longitude of the city
    api_url = 'https://api.api-ninjas.com/v1/city?name={}'.format(city)
    response = requests.get(api_url, headers={'X-Api-Key': API_NINJAS_API_KEY})
    lat = response.json()[0]['latitude']
    lon = response.json()[0]['longitude']

    # Now get the weather
    api_url = f'https://api.api-ninjas.com/v1/weather?lat={lat}&lon={lon}'
    response = requests.get(api_url, headers={'X-Api-Key': API_NINJAS_API_KEY})
    temp = response.json()['temp']

    return f'Current temperature in {city} is {temp} degrees'

# Create and Run AutoGen Agents

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.task import Console, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [6]:
async def main(city: str) -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=client,
        tools=[get_temp],
    )

    # Define termination condition
    termination = TextMentionTermination("TERMINATE")

    # Define a team
    agent_team = RoundRobinGroupChat([weather_agent], termination_condition=termination)

    # Run the team and stream messages to the console
    stream = agent_team.run_stream(task=f"What is current temperature in {city}?")
    await Console(stream)


In [7]:
await main(city='Paris')

---------- user ----------
What is current temperature in Paris?
---------- weather_agent ----------
[FunctionCall(id='call_5dhocGc4YgWXkkVQugA10ncp', arguments='{"city":"Paris"}', name='get_temp')]
[Prompt tokens: 78, Completion tokens: 14]
---------- weather_agent ----------
[FunctionExecutionResult(content='Current temperature in Paris is 5 degrees', call_id='call_5dhocGc4YgWXkkVQugA10ncp')]
---------- weather_agent ----------
The current temperature in Paris is 5 degrees.
[Prompt tokens: 84, Completion tokens: 11]
---------- weather_agent ----------
TERMINATE
[Prompt tokens: 128, Completion tokens: 4]
---------- Summary ----------
Number of messages: 5
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 290
Total completion tokens: 29
Duration: 40.91 seconds
